In [2]:

import torch 
from torch import nn

# ===========
# fake data randomly 
# this part is not important 
batch_size = 1
seq_len = 48
hidden_size = 1024
num_labels = 6
vector_utterances_without_context = torch.rand(batch_size, seq_len, hidden_size)
y_hat = torch.rand(batch_size, seq_len, num_labels)
intra_speaker_masked = (torch.rand(batch_size, seq_len, seq_len) > -0.5)
intra_speaker_masked[:, :seq_len-5, :seq_len-5] = False
padding_utterance_masked =  torch.rand(batch_size, seq_len) > -0.5
padding_utterance_masked[:, :seq_len-5] = False
# ===========

# model init 
d_model = hidden_size
inter_speaker_modeling = nn.MultiheadAttention(int(d_model/2), num_heads=8, dropout=0.2, batch_first=True)
inter_q =   nn.Linear(d_model, int(d_model/2))  
inter_k =   nn.Linear(d_model,int(d_model/2))  
inter_v =   nn.Linear(d_model,int(d_model/2))  
inter_speaker_2_output_layer =   nn.Linear(int(d_model/2), num_labels)  

# forward model 
q_vector = inter_q(vector_utterances_without_context)
k_vector = inter_k(vector_utterances_without_context)
v_vector = inter_v(vector_utterances_without_context)
vector_fused_by_inter_speaker, attentions = inter_speaker_modeling(q_vector, k_vector, v_vector, attn_mask=(intra_speaker_masked).repeat(8,1,1), key_padding_mask=padding_utterance_masked)
output_vector = inter_speaker_2_output_layer(vector_fused_by_inter_speaker)
output_vector = output_vector.reshape(batch_size*seq_len, -1)

# output shape 
# incoportate `output_vector` to the `y_hat`
y_hat = y_hat +  output_vector

print(output_vector)
print(output_vector.shape)

tensor([[-0.0938, -0.0957,  0.1413,  0.0250,  0.0825, -0.0657],
        [-0.1028, -0.0951,  0.1497,  0.0174,  0.0868, -0.0748],
        [-0.1163, -0.0932,  0.1367,  0.0185,  0.0929, -0.0708],
        [-0.1007, -0.0970,  0.1344,  0.0198,  0.0824, -0.0725],
        [-0.0969, -0.0904,  0.1369,  0.0130,  0.0827, -0.0707],
        [-0.1021, -0.0922,  0.1431,  0.0180,  0.0816, -0.0694],
        [-0.0981, -0.0931,  0.1511,  0.0196,  0.0763, -0.0704],
        [-0.1056, -0.0969,  0.1367,  0.0151,  0.0864, -0.0753],
        [-0.0931, -0.0974,  0.1386,  0.0211,  0.0823, -0.0684],
        [-0.1032, -0.1034,  0.1358,  0.0143,  0.0815, -0.0741],
        [-0.1083, -0.1003,  0.1428,  0.0124,  0.0856, -0.0744],
        [-0.1136, -0.1055,  0.1392,  0.0131,  0.0886, -0.0791],
        [-0.1104, -0.0871,  0.1386,  0.0184,  0.0853, -0.0697],
        [-0.1096, -0.0999,  0.1291,  0.0138,  0.0883, -0.0726],
        [-0.1092, -0.1001,  0.1374,  0.0208,  0.0865, -0.0692],
        [-0.0998, -0.0983,  0.1441,  0.0

/home/phuongnm/deeplearning_tutorial/env_py38/lib/python3.8/site-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
